In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
# import pixiedust                   ##debuger
# import glob, os

In [2]:
all_stim = pd.read_csv('2018_03_07_13_40_17_stim.csv', sep=';')

In [3]:
responses = pd.read_csv('2018_03_07_13_40_17.csv', sep=';', encoding = "ISO-8859-1")

# Feature task

In [4]:
#clean feature response with "bad response"
resp_feat = responses[responses["FeatConj"] == 0]
bad_feat = resp_feat.loc[(resp_feat['stType'] == ' distractor2 ') | (resp_feat['stType'] == ' distractor1 '), 
                         'trialN'].tolist()
resp_feat = resp_feat[~resp_feat['trialN']
          .isin(bad_feat)]
resp_feat = resp_feat.reset_index(drop=True)
resp_feat.head()

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,col,row,touchX,touchY,touchDist,targDist,touchN,error,switches,changeScreen
0,24,0,NaN,0,0,b/y,0,40,50,0.0,...,0.0,5.0,140.5,505.5,0.000000,0.000000,1,0,2,0
1,24,0,NaN,0,0,b/y,0,40,50,0.0,...,1.0,4.0,210.5,422.5,105.261579,114.687740,2,0,0,0
2,24,0,NaN,0,0,b/y,0,40,50,0.0,...,0.0,1.0,133.5,189.5,234.175682,244.654289,3,0,1,0
3,24,0,NaN,0,0,b/y,0,40,50,0.0,...,2.0,1.0,294.5,216.0,162.098735,151.908948,4,0,1,0
4,24,0,NaN,0,0,b/y,0,40,50,0.0,...,3.0,0.0,408.0,130.5,132.983082,146.506320,5,0,1,0


In [5]:
resp_feat1 = resp_feat.rename(columns={'stType': 'type', 
                                          'stPosX': 'pos_x_r', 
                                          'stPosY': 'pos_y_r',
                                         'trialN': 'trial_r',
                                      'participantN': 'p_n'})
resp_feat1['pos_x_r'] = np.around(resp_feat1['pos_x_r'], decimals=2)
resp_feat1['pos_y_r'] = np.around(resp_feat1['pos_y_r'], decimals=2)
resp_feat['resp_pos'] = resp_feat1.apply(lambda x: list([x['pos_x_r'], x['pos_y_r']]) , axis=1)
resp_feat1['pos_r'] = list(zip(resp_feat1['pos_x_r'], resp_feat1['pos_y_r']))

resp_feat_c = resp_feat1.loc[:, resp_feat1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n',
                                                                'pos_x_r',
                                                                'pos_y_r'])]

resp_feat1 = resp_feat1.loc[:, resp_feat1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n'])]

resp_feat1 = resp_feat1.reset_index(drop=True)
resp_feat1.type = resp_feat1.type.str.strip()

In [6]:
resp_feat_c

,p_n,trial_r,type,pos_x_r,pos_y_r,pos_r
0,24,0,target2,140.73,505.90,"(140.73, 505.9)"
1,24,0,target2,223.86,426.89,"(223.86, 426.89)"
2,24,0,target1,142.42,196.19,"(142.42, 196.19)"
3,24,0,target2,293.52,211.91,"(293.52, 211.91)"
4,24,0,target1,408.57,121.20,"(408.57, 121.2)"
...,...,...,...,...,...,...
195,24,9,target2,870.54,247.96,"(870.54, 247.96)"
196,24,9,target2,887.70,563.17,"(887.7, 563.17)"
197,24,9,target1,876.91,637.65,"(876.91, 637.65)"
198,24,9,target2,748.82,417.71,"(748.82, 417.71)"


In [7]:
resp_feat_c['x1'] = resp_feat_c['pos_x_r'].apply(lambda x: x + 100)
resp_feat_c['x2'] = resp_feat_c['pos_x_r'].apply(lambda x: x - 100)
resp_feat_c['y1'] = resp_feat_c['pos_y_r'].apply(lambda x: x + 100)
resp_feat_c['y2'] = resp_feat_c['pos_y_r'].apply(lambda x: x - 100)

In [8]:
resp_feat_c

,p_n,trial_r,type,pos_x_r,pos_y_r,pos_r,x1,x2,y1,y2
0,24,0,target2,140.73,505.90,"(140.73, 505.9)",240.73,40.73,605.90,405.90
1,24,0,target2,223.86,426.89,"(223.86, 426.89)",323.86,123.86,526.89,326.89
2,24,0,target1,142.42,196.19,"(142.42, 196.19)",242.42,42.42,296.19,96.19
3,24,0,target2,293.52,211.91,"(293.52, 211.91)",393.52,193.52,311.91,111.91
4,24,0,target1,408.57,121.20,"(408.57, 121.2)",508.57,308.57,221.20,21.20
...,...,...,...,...,...,...,...,...,...,...
195,24,9,target2,870.54,247.96,"(870.54, 247.96)",970.54,770.54,347.96,147.96
196,24,9,target2,887.70,563.17,"(887.7, 563.17)",987.70,787.70,663.17,463.17
197,24,9,target1,876.91,637.65,"(876.91, 637.65)",976.91,776.91,737.65,537.65
198,24,9,target2,748.82,417.71,"(748.82, 417.71)",848.82,648.82,517.71,317.71


In [9]:
feat_r = resp_feat1.values.tolist()

In [10]:
#only useful df of all stimuli for feature task
all_feat = all_stim[all_stim["FeatConj"] == 0]
feat_target = all_feat[(all_feat['stType'] == 'target1') 
                       | (all_feat['stType'] == 'target2')]
feat_distr = all_feat[(all_feat['stType'] == 'distractor1') 
                      | (all_feat['stType'] == 'distractor2')]

In [11]:
#useful targets for feature (only 3 columns)
feat_rest_t = feat_target[~feat_target['trialN']
          .isin(bad_feat)]
feat_rest_t = feat_rest_t.rename(columns={'stType': 'target', 
                                          'posX': 'pos_x_t', 
                                          'posY': 'pos_y_t',
                                         'trialN': 'trial_t',
                                         'participantN': 'p_n_t'})
feat_rest_t['pos_x_t'] = np.around(feat_rest_t['pos_x_t'], decimals=2)
feat_rest_t['pos_y_t'] = np.around(feat_rest_t['pos_y_t'], decimals=2)
feat_rest_t['pos_t'] = list(zip(feat_rest_t['pos_x_t'], feat_rest_t['pos_y_t']))
feat_rest_t = feat_rest_t.loc[:, feat_rest_t.columns.intersection(['target',
                                                                   'pos_t',
                                                                   'trial_t',
                                                                  'p_n_t'])]
feat_rest_t = feat_rest_t.reset_index(drop=True)

In [12]:
#numbers of trials in list
trials = feat_rest_t['trial_t'].unique()
trials = trials.astype(np.int32).tolist()

In [13]:
participant = feat_rest_t['p_n_t'].unique()
participant = participant.astype(np.int32).tolist()

In [14]:
dfs_tar_f = dict(tuple(feat_rest_t.groupby('target')))
feat_t1 = dfs_tar_f['target1']
feat_t2 = dfs_tar_f['target2']

In [15]:
t1 = dict(tuple(feat_t1.groupby('trial_t')))
t2 = dict(tuple(feat_t2.groupby('trial_t')))

In [16]:
def closest_node(node, nodes):
    indexArr = np.argwhere(nodes == node)
    nodes = np.delete(nodes,indexArr, 0)
    closest_index = distance.cdist([node], nodes, 'euclidean').argmin()
    return nodes[closest_index]

In [17]:
# %%pixie_debugger 
closest_same = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    if feat_r[j][2] == 'target1':
                        closest_t1 = closest_node(feat_r[j][3], t1_arr)                          
                        closest_same.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(feat_r[j][3], t2_arr)                         
                        closest_same.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1
               

In [18]:
resp_feat['closest_same_target'] = closest_same

In [19]:
closest_other = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    if feat_r[j][2] == 'target2':
                        closest_t1 = closest_node(feat_r[j][3], t1_arr)                          
                        closest_other.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(feat_r[j][3], t2_arr)                         
                        closest_other.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [20]:
resp_feat['closest_other_taget'] = closest_other

In [21]:
feat_rest_d = feat_distr[~feat_distr['trialN']
          .isin(bad_feat)]
feat_rest_d = feat_rest_d.rename(columns={'stType': 'distractor', 
                                          'posX': 'pos_x_d', 
                                          'posY': 'pos_y_d',
                                         'trialN': 'trial_d',
                                         'participantN': 'p_n_d'})
feat_rest_d['pos_x_d'] = np.around(feat_rest_d['pos_x_d'], decimals=2)
feat_rest_d['pos_y_d'] = np.around(feat_rest_d['pos_y_d'], decimals=2)
feat_rest_d['pos_d'] = list(zip(feat_rest_d['pos_x_d'], feat_rest_d['pos_y_d']))
feat_rest_d = feat_rest_d.loc[:, feat_rest_d.columns.intersection(['distractor',
                                                                   'pos_d',
                                                                   'trial_d',
                                                                  'p_n_d'])]
feat_rest_d = feat_rest_d.reset_index(drop=True)

In [22]:
dfs_d_f = dict(tuple(feat_rest_d.groupby('distractor')))
feat_d1 = dfs_d_f['distractor1']
feat_d2 = dfs_d_f['distractor2']
d1 = dict(tuple(feat_d1.groupby('trial_d')))
d2 = dict(tuple(feat_d2.groupby('trial_d')))

In [23]:
# closest blue dot
closest_same_d = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        d1_arr = np.array(d1.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    closest_d1 = closest_node(feat_r[j][3], d1_arr)                          
                    closest_same_d.append(closest_d1.tolist())         
                else:
                    i += 1
            else:
                n += 1

In [24]:
resp_feat['closest_blue_dot'] = closest_same_d

In [25]:
# closest yellow dot
closest_d2 = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        d1_arr = np.array(d2.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    c_d1 = closest_node(feat_r[j][3], d1_arr)                          
                    closest_d2.append(c_d1.tolist())         
                else:
                    i += 1
            else:
                n += 1

In [26]:
resp_feat['closest_yellow_dot'] = closest_d2

In [27]:
resp_feat_c

,p_n,trial_r,type,pos_x_r,pos_y_r,pos_r,x1,x2,y1,y2
0,24,0,target2,140.73,505.90,"(140.73, 505.9)",240.73,40.73,605.90,405.90
1,24,0,target2,223.86,426.89,"(223.86, 426.89)",323.86,123.86,526.89,326.89
2,24,0,target1,142.42,196.19,"(142.42, 196.19)",242.42,42.42,296.19,96.19
3,24,0,target2,293.52,211.91,"(293.52, 211.91)",393.52,193.52,311.91,111.91
4,24,0,target1,408.57,121.20,"(408.57, 121.2)",508.57,308.57,221.20,21.20
...,...,...,...,...,...,...,...,...,...,...
195,24,9,target2,870.54,247.96,"(870.54, 247.96)",970.54,770.54,347.96,147.96
196,24,9,target2,887.70,563.17,"(887.7, 563.17)",987.70,787.70,663.17,463.17
197,24,9,target1,876.91,637.65,"(876.91, 637.65)",976.91,776.91,737.65,537.65
198,24,9,target2,748.82,417.71,"(748.82, 417.71)",848.82,648.82,517.71,317.71


# Crowding_feat

In [28]:
def close_or_not(x1, x2, y1, y2, lst1, lst2):
    count = 0
    for i in range(0, len(lst1)):
        if x2 < lst1[i] < x1 and y2 < lst2[i] < y1:
            count += 1
    return count

In [29]:
feat_r = resp_feat_c.values.tolist()

In [30]:
feat_r

[[24,
  0,
  'target2',
  140.73,
  505.9,
  (140.73, 505.9),
  240.73,
  40.72999999999999,
  605.9,
  405.9],
 [24,
  0,
  'target2',
  223.86,
  426.89,
  (223.86, 426.89),
  323.86,
  123.86000000000001,
  526.89,
  326.89],
 [24,
  0,
  'target1',
  142.42,
  196.19,
  (142.42, 196.19),
  242.42,
  42.41999999999999,
  296.19,
  96.19],
 [24,
  0,
  'target2',
  293.52,
  211.91,
  (293.52, 211.91),
  393.52,
  193.51999999999998,
  311.90999999999997,
  111.91],
 [24,
  0,
  'target1',
  408.57,
  121.2,
  (408.57, 121.2),
  508.57,
  308.57,
  221.2,
  21.200000000000003],
 [24,
  0,
  'target2',
  452.4,
  361.28,
  (452.4, 361.28),
  552.4,
  352.4,
  461.28,
  261.28],
 [24,
  0,
  'target2',
  404.98,
  422.65,
  (404.98, 422.65),
  504.98,
  304.98,
  522.65,
  322.65],
 [24,
  0,
  'target2',
  383.8,
  499.31,
  (383.8, 499.31),
  483.8,
  283.8,
  599.31,
  399.31],
 [24,
  0,
  'target1',
  397.15,
  567.01,
  (397.15, 567.01),
  497.15,
  297.15,
  667.01,
  467.01],
 

In [31]:
feat_t1[['pos_x', 'pos_y']] = pd.DataFrame(feat_t1['pos_t'].tolist(), index=feat_t1.index) 
feat_t2[['pos_x', 'pos_y']] = pd.DataFrame(feat_t1['pos_t'].tolist(), index=feat_t1.index)

In [32]:
t1 = dict(tuple(feat_t1.groupby('trial_t')))
t2 = dict(tuple(feat_t2.groupby('trial_t')))

In [33]:
crowding = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(feat_r)):
            if part == feat_r[j][0]:
                if n_tr == feat_r[j][1]:
                    if feat_r[j][2] == 'target1':
                        closest_t1 = close_or_not(feat_r[j][6],
                                                  feat_r[j][7],
                                                  feat_r[j][8],
                                                  feat_r[j][9],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not(feat_r[j][6],
                                                  feat_r[j][7],
                                                  feat_r[j][8],
                                                  feat_r[j][9],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [34]:
resp_feat['crowd'] = crowding

# Conjunction task

In [35]:
resp_conj = responses[responses["FeatConj"] == 1]
bad_conj = resp_conj.loc[(resp_conj['stType'] == ' distractor2 ') | (resp_conj['stType'] == ' distractor1 '), 
                         'trialN'].tolist()
resp_conj = resp_conj[~resp_conj['trialN']
          .isin(bad_conj)]
resp_conj = resp_conj.reset_index(drop=True)

In [36]:
resp_conj1 = resp_conj.rename(columns={'stType': 'type', 
                                          'stPosX': 'pos_x_r', 
                                          'stPosY': 'pos_y_r',
                                         'trialN': 'trial_r',
                                      'participantN': 'p_n'})
resp_conj1['pos_x_r'] = np.around(resp_conj1['pos_x_r'], decimals=2)
resp_conj1['pos_y_r'] = np.around(resp_conj1['pos_y_r'], decimals=2)
resp_conj['resp_pos'] = resp_conj1.apply(lambda x: list([x['pos_x_r'], x['pos_y_r']]) , axis=1)
resp_conj1['pos_r'] = list(zip(resp_conj1['pos_x_r'], resp_conj1['pos_y_r']))

resp_conj_c = resp_conj1.loc[:, resp_conj1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n',
                                                               'pos_x_r',
                                                               'pos_y_r'])]

resp_conj1 = resp_conj1.loc[:, resp_conj1.columns.intersection(['type', 
                                                             'pos_r',
                                                             'trial_r',
                                                               'p_n'])]

resp_conj1 = resp_conj1.reset_index(drop=True)
resp_conj1.type = resp_conj1.type.str.strip()

In [37]:
resp_conj_c['x1'] = resp_conj_c['pos_x_r'].apply(lambda x: x + 100)
resp_conj_c['x2'] = resp_conj_c['pos_x_r'].apply(lambda x: x - 100)
resp_conj_c['y1'] = resp_conj_c['pos_y_r'].apply(lambda x: x + 100)
resp_conj_c['y2'] = resp_conj_c['pos_y_r'].apply(lambda x: x - 100)

In [38]:
conj_r = resp_conj1.values.tolist()
## [prtN, trialN, target_type, pos]

In [39]:
#only useful df of all stimuli for conjunction task
all_conj = all_stim[all_stim["FeatConj"] == 1]
conj_target = all_conj[(all_conj['stType'] == 'target1') |
                       (all_conj['stType'] == 'target2')]
conj_distr = all_conj[(all_conj['stType'] == 'distractor1') |
                      (all_conj['stType'] == 'distractor2')]

In [40]:
#useful targets for conjunction
conj_rest_t = conj_target[~conj_target['trialN']
          .isin(bad_conj)]
conj_rest_t = conj_rest_t.rename(columns={'stType': 'target', 
                                          'posX': 'pos_x_t', 
                                          'posY': 'pos_y_t',
                                         'trialN': 'trial_t',
                                         'participantN': 'p_n_t'})
conj_rest_t['pos_x_t'] = np.around(conj_rest_t['pos_x_t'], decimals=2)
conj_rest_t['pos_y_t'] = np.around(conj_rest_t['pos_y_t'], decimals=2)
conj_rest_t['pos_t'] = list(zip(conj_rest_t['pos_x_t'], conj_rest_t['pos_y_t']))
conj_rest_t = conj_rest_t.loc[:, conj_rest_t.columns.intersection(['target',
                                                                   'pos_t',
                                                                   'trial_t',
                                                                  'p_n_t'])]
conj_rest_t = conj_rest_t.reset_index(drop=True)

In [41]:
trials = conj_rest_t['trial_t'].unique()
trials = trials.astype(np.int32).tolist()

In [42]:
participant = conj_rest_t['p_n_t'].unique()
participant = participant.astype(np.int32).tolist()

In [43]:
dfs_tar_c = dict(tuple(conj_rest_t.groupby('target')))
conj_t1 = dfs_tar_c['target1']
conj_t2 = dfs_tar_c['target2']

In [44]:
t1 = dict(tuple(conj_t1.groupby('trial_t')))
t2 = dict(tuple(conj_t2.groupby('trial_t')))

In [45]:
closest_same = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target1':
                        closest_t1 = closest_node(conj_r[j][3], t1_arr)                          
                        closest_same.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(conj_r[j][3], t2_arr)                         
                        closest_same.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [46]:
resp_conj['closest_same_target'] = closest_same

In [47]:
closest_other = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(t1.get(trials[i])['pos_t'].values.tolist())
        t2_arr = np.array(t2.get(trials[i])['pos_t'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target2':
                        closest_t1 = closest_node(conj_r[j][3], t1_arr)                          
                        closest_other.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(conj_r[j][3], t2_arr)                         
                        closest_other.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [48]:
resp_conj['closest_other_target'] = closest_other

In [49]:
#useful distractors for conjunction
conj_rest_d = conj_distr[~conj_distr['trialN']
          .isin(bad_conj)]
conj_rest_d = conj_rest_d.rename(columns={'stType': 'distractor', 
                                          'posX': 'pos_x_d', 
                                          'posY': 'pos_y_d',
                                         'trialN': 'trial_d',
                                         'participantN': 'p_n_d'})
conj_rest_d['pos_x_d'] = np.around(conj_rest_d['pos_x_d'], decimals=2)
conj_rest_d['pos_y_d'] = np.around(conj_rest_d['pos_y_d'], decimals=2)
conj_rest_d['pos_d'] = list(zip(conj_rest_d['pos_x_d'], conj_rest_d['pos_y_d']))
conj_rest_d = conj_rest_d.loc[:, conj_rest_d.columns.intersection(['distractor',
                                                                   'pos_d',
                                                                   'trial_d',
                                                                  'p_n_d'])]
conj_rest_d = conj_rest_d.reset_index(drop=True)

In [50]:
dfs_d_c = dict(tuple(conj_rest_d.groupby('distractor')))
conj_d1 = dfs_d_c['distractor1']
conj_d2 = dfs_d_c['distractor2']
d1 = dict(tuple(conj_d1.groupby('trial_d')))
d2 = dict(tuple(conj_d2.groupby('trial_d')))

In [51]:
same_color = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(d1.get(trials[i])['pos_d'].values.tolist())
        t2_arr = np.array(d2.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target1':
                        closest_d1 = closest_node(conj_r[j][3], t1_arr)                          
                        same_color.append(closest_d1.tolist())         
                    else:
                        closest_d2 = closest_node(conj_r[j][3], t2_arr)                         
                        same_color.append(closest_d2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [52]:
resp_conj['closest_dist_same_color'] = same_color

In [53]:
same_shape = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr = np.array(d1.get(trials[i])['pos_d'].values.tolist())
        t2_arr = np.array(d2.get(trials[i])['pos_d'].values.tolist())
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target2':
                        closest_t1 = closest_node(conj_r[j][3], t1_arr)                          
                        same_shape.append(closest_t1.tolist())         
                    else:
                        closest_t2 = closest_node(conj_r[j][3], t2_arr)                         
                        same_shape.append(closest_t2.tolist())
                else:
                    i += 1
            else:
                n += 1

In [54]:
resp_conj['closest_dist_same_shape'] = same_shape

# Crowding_conj

In [55]:
conj_r = resp_conj_c.values.tolist()

In [56]:
conj_t1

,p_n_t,trial_t,target,pos_t
0,24,0,target1,"(787.56, 92.8)"
1,24,0,target1,"(395.4, 184.82)"
2,24,0,target1,"(376.85, 105.67)"
3,24,0,target1,"(189.5, 193.08)"
4,24,0,target1,"(572.57, 598.81)"
...,...,...,...,...
265,24,13,target1,"(823.18, 251.21)"
266,24,13,target1,"(364.08, 597.68)"
267,24,13,target1,"(109.04, 669.07)"
268,24,13,target1,"(194.56, 508.63)"


In [57]:
conj_t1[['pos_x', 'pos_y']] = pd.DataFrame(conj_t1['pos_t'].tolist(), index=conj_t1.index) 
conj_t2[['pos_x', 'pos_y']] = pd.DataFrame(conj_t2['pos_t'].tolist(), index=conj_t1.index)

In [58]:
t1 = dict(tuple(conj_t1.groupby('trial_t')))
t2 = dict(tuple(conj_t2.groupby('trial_t')))

In [59]:
crowding = []
for n in range(0, len(participant)):
    part = participant[n]
    for i in range(0, len(trials)):
        n_tr = trials[i]
        t1_arr_x = t1.get(trials[i])['pos_x'].values.tolist()
        t1_arr_y = t1.get(trials[i])['pos_y'].values.tolist()
        t2_arr_x = t2.get(trials[i])['pos_x'].values.tolist()
        t2_arr_y = t2.get(trials[i])['pos_y'].values.tolist()
        
        for j in range(0, len(conj_r)):
            if part == conj_r[j][0]:
                if n_tr == conj_r[j][1]:
                    if conj_r[j][2] == 'target1':
                        closest_t1 = close_or_not(conj_r[j][6],
                                                  conj_r[j][7],
                                                  conj_r[j][8],
                                                  conj_r[j][9],
                                                  t1_arr_x, t1_arr_y)                          
                        crowding.append(closest_t1)         
                    else:
                        closest_t2 = close_or_not(conj_r[j][6],
                                                  conj_r[j][7],
                                                  conj_r[j][8],
                                                  conj_r[j][9],
                                                  t2_arr_x, t2_arr_y)                          
                        crowding.append(closest_t2)
                else:
                    i += 1
            else:
                n += 1

In [60]:
resp_conj['crowd'] = crowding

In [61]:
resp_feat.head()

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,touchN,error,switches,changeScreen,resp_pos,closest_same_target,closest_other_taget,closest_blue_dot,closest_yellow_dot,crowd
0,24,0,NaN,0,0,b/y,0,40,50,0.0,...,1,0,2,0,"[140.73, 505.9]","[223.86, 426.89]","[397.15, 567.01]","[132.37, 597.88]","[103.23, 400.17]",0
1,24,0,NaN,0,0,b/y,0,40,50,0.0,...,2,0,0,0,"[223.86, 426.89]","[140.73, 505.9]","[397.15, 567.01]","[292.56, 585.59]","[103.23, 400.17]",0
2,24,0,NaN,0,0,b/y,0,40,50,0.0,...,3,0,1,0,"[142.42, 196.19]","[408.57, 121.2]","[293.52, 211.91]","[237.77, 176.48]","[123.47, 273.73]",1
3,24,0,NaN,0,0,b/y,0,40,50,0.0,...,4,0,1,0,"[293.52, 211.91]","[223.86, 426.89]","[408.57, 121.2]","[237.77, 176.48]","[218.65, 252.12]",0
4,24,0,NaN,0,0,b/y,0,40,50,0.0,...,5,0,1,0,"[408.57, 121.2]","[142.42, 196.19]","[293.52, 211.91]","[546.38, 120.63]","[488.12, 115.99]",1


In [62]:
resp_feat.columns

Index(['participantN', 'age', 'gender', 'trialN', 'task', 'condition',
       'FeatConj', 'setSize', 'proportion', 'timelimit', 'stType', 'imgName',
       'timeTS', 'timeRel', 'runLength', 'runTest', 'touchTS', 'runN', 'runNH',
       'stPosX', 'stPosY', 'col', 'row', 'touchX', 'touchY', 'touchDist',
       'targDist', 'touchN', 'error', 'switches', 'changeScreen', 'resp_pos',
       'closest_same_target', 'closest_other_taget', 'closest_blue_dot',
       'closest_yellow_dot', 'crowd'],
      dtype='object')

In [63]:
resp_feat

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,touchN,error,switches,changeScreen,resp_pos,closest_same_target,closest_other_taget,closest_blue_dot,closest_yellow_dot,crowd
0,24,0,NaN,0,0,b/y,0,40,50,0.0,...,1,0,2,0,"[140.73, 505.9]","[223.86, 426.89]","[397.15, 567.01]","[132.37, 597.88]","[103.23, 400.17]",0
1,24,0,NaN,0,0,b/y,0,40,50,0.0,...,2,0,0,0,"[223.86, 426.89]","[140.73, 505.9]","[397.15, 567.01]","[292.56, 585.59]","[103.23, 400.17]",0
2,24,0,NaN,0,0,b/y,0,40,50,0.0,...,3,0,1,0,"[142.42, 196.19]","[408.57, 121.2]","[293.52, 211.91]","[237.77, 176.48]","[123.47, 273.73]",1
3,24,0,NaN,0,0,b/y,0,40,50,0.0,...,4,0,1,0,"[293.52, 211.91]","[223.86, 426.89]","[408.57, 121.2]","[237.77, 176.48]","[218.65, 252.12]",0
4,24,0,NaN,0,0,b/y,0,40,50,0.0,...,5,0,1,0,"[408.57, 121.2]","[142.42, 196.19]","[293.52, 211.91]","[546.38, 120.63]","[488.12, 115.99]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,24,0,NaN,9,0,b/y,0,40,50,0.0,...,17,0,1,0,"[870.54, 247.96]","[748.82, 417.71]","[822.67, 291.07]","[799.55, 192.92]","[809.26, 345.62]",0
196,24,0,NaN,9,0,b/y,0,40,50,0.0,...,18,0,0,0,"[887.7, 563.17]","[748.82, 417.71]","[876.91, 637.65]","[803.22, 639.21]","[809.26, 345.62]",0
197,24,0,NaN,9,0,b/y,0,40,50,0.0,...,19,0,1,0,"[876.91, 637.65]","[822.67, 291.07]","[887.7, 563.17]","[803.22, 639.21]","[809.26, 345.62]",1
198,24,0,NaN,9,0,b/y,0,40,50,0.0,...,20,0,1,0,"[748.82, 417.71]","[637.14, 262.6]","[665.62, 437.26]","[653.14, 494.47]","[739.4, 325.14]",0


In [64]:
resp_conj.head()

,participantN,age,gender,trialN,task,condition,FeatConj,setSize,proportion,timelimit,...,touchN,error,switches,changeScreen,resp_pos,closest_same_target,closest_other_target,closest_dist_same_color,closest_dist_same_shape,crowd
0,24,0,NaN,0,0,rs/gd,1,40,50,0.0,...,1,0,2,0,"[293.08, 136.04]","[376.85, 105.67]","[380.41, 287.86]","[212.4, 97.68]","[121.07, 214.95]",2
1,24,0,NaN,0,0,rs/gd,1,40,50,0.0,...,2,0,1,0,"[484.6, 105.44]","[618.27, 264.44]","[376.85, 105.67]","[629.34, 104.7]","[712.51, 92.17]",0
2,24,0,NaN,0,0,rs/gd,1,40,50,0.0,...,3,0,1,0,"[376.85, 105.67]","[293.08, 136.04]","[484.6, 105.44]","[212.4, 97.68]","[629.34, 104.7]",3
3,24,0,NaN,0,0,rs/gd,1,40,50,0.0,...,4,0,0,0,"[395.4, 184.82]","[376.85, 105.67]","[380.41, 287.86]","[286.92, 267.69]","[373.65, 362.28]",2
4,24,0,NaN,0,0,rs/gd,1,40,50,0.0,...,5,0,0,0,"[189.5, 193.08]","[293.08, 136.04]","[122.24, 367.0]","[212.4, 97.68]","[121.07, 214.95]",1


In [65]:
#resp_feat.to_csv (r'C:\Users\User\Desktop\feature_task.csv', index = False,  mode='a', header=False)

In [66]:
#resp_conj.to_csv (r'C:\Users\User\Desktop\conjunction_task.csv', index = False, mode='a', header=False)